In [3]:
import pandas as pd

# Assuming the file is a CSV for this example. Adjust the path and file reading method accordingly if different.
file_path = 'C:/Users/nate.marko/Downloads/TroutData.csv'
trout_data = pd.read_csv(file_path)

# Display the first few rows to confirm successful loading
print(trout_data.head())


                  pitch_id_raw pitch_type  zone    description events  \
0  413649_545361_433587_1_24_4         SL  14.0  called_strike    NaN   
1   413649_545361_433587_1_2_1         SI  13.0           foul    NaN   
2  413649_545361_433587_1_42_6         CU  11.0           ball    NaN   
3  413649_545361_433587_2_24_4         CH  13.0           foul    NaN   
4   413649_545361_433587_2_2_1         FF   1.0  called_strike    NaN   

   launch_speed  launch_angle  bearing_angle  
0          90.7          28.7            NaN  
1           NaN           NaN            NaN  
2           NaN           NaN            NaN  
3           NaN           NaN            NaN  
4           NaN           NaN            NaN  


In [4]:
# Filtering the data based on the description
filtered_data = trout_data[trout_data['description'].isin(['hit_into_play_score', 'hit_into_play_no_out', 'hit_into_play'])]

# Display the filtered data
print(filtered_data.head())


                   pitch_id_raw pitch_type  zone           description  \
17   413649_545361_433587_8_2_1         SI   5.0   hit_into_play_score   
23  413664_545361_500721_2_53_8         FF   8.0  hit_into_play_no_out   
28   413664_545361_572020_2_2_1         FF  13.0         hit_into_play   
36  413664_545361_572020_6_23_4         KC  14.0         hit_into_play   
46  413671_545361_547874_1_35_5         SI   8.0         hit_into_play   

       events  launch_speed  launch_angle  bearing_angle  
17   home_run         112.1          31.5        -6.8320  
23     double         111.5          15.6         9.1710  
28  field_out          94.9          29.6        27.5497  
36  field_out          71.9          35.6         1.3609  
46  field_out          90.2          -3.3       -24.1821  


In [14]:
filtered_data = filtered_data.dropna(subset=['launch_speed', 'launch_angle', 'bearing_angle'])


filtered_data[['launch_speed', 'launch_angle', 'bearing_angle']] = filtered_data[['launch_speed', 'launch_angle', 'bearing_angle']].round(0).astype(int)

# Grouping the data by the unique combinations of 'launch_speed', 'launch_angle', 'bearing_angle'
# and aggregating the counts for each type of hit outcome
hit_outcome_counts = filtered_data.groupby(['launch_speed', 'launch_angle', 'bearing_angle'])['events'].value_counts().unstack(fill_value=0)

# Since there are more event types than the ones we're interested in, we'll filter the columns after unstacking
desired_outcomes = ['single', 'double', 'triple', 'home_run']
hit_outcome_counts = hit_outcome_counts[desired_outcomes]

# Renaming columns for clarity after filtering to ensure we only have the columns we're interested in
hit_outcome_counts.columns = ['single_count', 'double_count', 'triple_count', 'homerun_count']

# Adding a column for the total count of each unique combination, including all events, not just hits
hit_outcome_counts['total_count'] = filtered_data.groupby(['launch_speed', 'launch_angle', 'bearing_angle']).size()

# Resetting index to make 'launch_speed', 'launch_angle', 'bearing_angle' columns again
hit_outcome_counts.reset_index(inplace=True)

# Display the new dataframe
print(hit_outcome_counts.head())


   launch_speed  launch_angle  bearing_angle  single_count  double_count  \
0            22           -23             40             0             0   
1            34            -4             32             0             0   
2            35            16            -45             0             0   
3            37            16            -11             1             0   
4            39           -52             -8             0             0   

   triple_count  homerun_count  total_count  
0             0              0            1  
1             0              0            1  
2             0              0            1  
3             0              0            1  
4             0              0            1  


In [19]:
# Calculating the sum of all hit counts for each combination
hit_outcome_counts['hit_sum'] = hit_outcome_counts[['single_count', 'double_count', 'triple_count', 'homerun_count']].sum(axis=1)


# Calculating the batting average for each combination
hit_outcome_counts['batting_average'] = hit_outcome_counts['hit_sum'] / hit_outcome_counts['total_count']

# Display the updated dataframe with the batting average column
print(hit_outcome_counts.head())


   launch_speed  launch_angle  bearing_angle  single_count  double_count  \
0            22           -23             40             0             0   
1            34            -4             32             0             0   
2            35            16            -45             0             0   
3            37            16            -11             1             0   
4            39           -52             -8             0             0   

   triple_count  homerun_count  total_count  hit_sum  batting_average  
0             0              0            1        0              0.0  
1             0              0            1        0              0.0  
2             0              0            1        0              0.0  
3             0              0            1        1              1.0  
4             0              0            1        0              0.0  


In [20]:
# Calculating the average of each outcome by dividing the total amount in those columns by total_count
hit_outcome_counts['average_single'] = hit_outcome_counts['single_count'] / hit_outcome_counts['total_count']
hit_outcome_counts['average_double'] = hit_outcome_counts['double_count'] / hit_outcome_counts['total_count']
hit_outcome_counts['average_triple'] = hit_outcome_counts['triple_count'] / hit_outcome_counts['total_count']
hit_outcome_counts['average_homerun'] = hit_outcome_counts['homerun_count'] / hit_outcome_counts['total_count']

# Display the updated dataframe with the average outcome columns
print(hit_outcome_counts.head())


   launch_speed  launch_angle  bearing_angle  single_count  double_count  \
0            22           -23             40             0             0   
1            34            -4             32             0             0   
2            35            16            -45             0             0   
3            37            16            -11             1             0   
4            39           -52             -8             0             0   

   triple_count  homerun_count  total_count  hit_sum  batting_average  \
0             0              0            1        0              0.0   
1             0              0            1        0              0.0   
2             0              0            1        0              0.0   
3             0              0            1        1              1.0   
4             0              0            1        0              0.0   

   average_single  average_double  average_triple  average_homerun  
0             0.0             0.0  